<a href="https://colab.research.google.com/github/pagnihot/pa-pub-test/blob/main/Portfolio_Risk_MC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
    from google.colab import auth
    GOOGLE_COLAB = True
except ImportError:
    GOOGLE_COLAB = False

In [56]:
if GOOGLE_COLAB:
    !pip install google-cloud-pubsub

In [55]:
if GOOGLE_COLAB:
    auth.authenticate_user()

AttributeError: ignored

In [57]:
from google import auth
from google.cloud import storage

import datetime, time
import subprocess
import argparse
import os, sys
import json

In [58]:
import pandas as pd
from pandas_datareader import data as pdr
import numpy as np
import datetime , time
from math import sqrt
from scipy.stats import norm
import yfinance as yf

In [59]:
STOCK="AMZN"
short_window = 40
long_window = 100
trading_days = 500
sim_num = 10
portfolio_stocks_list = ["GOOG","MSFT","NFLX","TSLA","IBM","AMZN",""]
portfolio_capital=100000
lookback_days=730

In [60]:
import uuid
sim_id = str(uuid.uuid4())

In [61]:
period_end=datetime.datetime.now()

In [62]:
period_start = period_end - datetime.timedelta(days=lookback_days)

In [63]:
yf.pdr_override()
stock_df = pdr.get_data_yahoo(STOCK,start=period_start, end=period_end)

[*********************100%%**********************]  1 of 1 completed


In [64]:
# Calculate the compound annual growth rate (CAGR) which
# will give us our mean return input (mu)
days = (stock_df.index[-1] - stock_df.index[0]).days
cagr = ((((stock_df['Adj Close'][-1]) / stock_df['Adj Close'][1])) ** (365.0/days)) - 1
mu = cagr

In [65]:
stock_df['Returns'] = stock_df['Adj Close'].pct_change()
vol = stock_df['Returns'].std()*sqrt(252)

In [66]:
initial_capital= float(portfolio_capital)

In [67]:
# Set up empty list to hold our ending values for each simulated price series
sim_result = []

# Set up empty list to hold portfolio value for each simulated price serries, this is the value of position['total']
portfolio_total = []

In [68]:
# Define Variables
start_price = stock_df['Adj Close'][-1] #starting stock price (i.e. last available real stock price)

In [69]:
from concurrent import futures
from google.cloud import pubsub_v1
from typing import Callable

In [70]:
# Define GCP Environment
project_id = "fsif-330919"
topic_id = "QU1"
subscription_id = f'{topic_id}-sub1'

subscriber = pubsub_v1.SubscriberClient()
subscription_path = subscriber.subscription_path(project_id, subscription_id)

publisher = pubsub_v1.PublisherClient()
topic_path = publisher.topic_path(project_id, topic_id)
publish_futures = []

In [71]:
def get_callback(publish_future: pubsub_v1.publisher.futures.Future, data: str) -> Callable[[pubsub_v1.publisher.futures.Future], None]:

  def pub_callback(publish_future: pubsub_v1.publisher.futures.Future) -> None:
    try:
      # Wait 60 seconds for the publish call to succeed.
      print(publish_future.result(timeout=60))
    except futures.TimeoutError:
      print(f"Publishing {data} timed out.")

  return pub_callback

In [72]:
def sub_callback(message: pubsub_v1.subscriber.message.Message) -> None:
  """Process received message"""
  print(f"Received message: ID={message.message_id} Data={message.data}")
  print(f"[{datetime.datetime.now()}] Processing: {message.message_id}")
  sim_data = json.loads(message.data)
  sim_num          = sim_data["sim_num"]
  sim_result.append(sim_data["stock_price"])
  portfolio_total.append(sim_data["portfolio_value"])
  print(f"[{datetime.datetime.now()}] Processed: {message.message_id}")
  message.ack()

In [74]:
for i in range(sim_num):
  data = f'{{ "sim_id": "{sim_id}", "sim_num": {i}, "stock": "{STOCK}", "short_window": {short_window}, "long_window": {long_window}, "trading_days": {trading_days}, "start_price": {start_price}, "lookback_days": {lookback_days}, "mu": {mu}, "vol": {vol}, "initial_capital": {initial_capital}  }}'
  print(data)
  publish_future = publisher.publish(topic_path, data.encode("utf-8"))
  # Non-blocking. Publish failures are handled in the callback function.
  publish_future.add_done_callback(get_callback(publish_future, data))
  publish_futures.append(publish_future)

print(f"Published {i} messages with error handler to {topic_path}.")


{ "sim_id": "992d348d-ce69-4a7f-a16f-a4006e6951ab", "sim_num": 0, "stock": "AMZN", "short_window": 40, "long_window": 100, "trading_days": 500, "start_price": 133.08999633789062, "lookback_days": 730, "mu": -0.10375197793841895, "vol": 0.42759959137708875, "initial_capital": 100000.0  }
{ "sim_id": "992d348d-ce69-4a7f-a16f-a4006e6951ab", "sim_num": 1, "stock": "AMZN", "short_window": 40, "long_window": 100, "trading_days": 500, "start_price": 133.08999633789062, "lookback_days": 730, "mu": -0.10375197793841895, "vol": 0.42759959137708875, "initial_capital": 100000.0  }
{ "sim_id": "992d348d-ce69-4a7f-a16f-a4006e6951ab", "sim_num": 2, "stock": "AMZN", "short_window": 40, "long_window": 100, "trading_days": 500, "start_price": 133.08999633789062, "lookback_days": 730, "mu": -0.10375197793841895, "vol": 0.42759959137708875, "initial_capital": 100000.0  }
{ "sim_id": "992d348d-ce69-4a7f-a16f-a4006e6951ab", "sim_num": 3, "stock": "AMZN", "short_window": 40, "long_window": 100, "trading_days

In [ ]:
streaming_pull_future = subscriber.subscribe(subscription_path, callback=sub_callback)
print(f"Pulling messages from {subscription_path}..., in project:{project_id}")

with subscriber:
  try:
    streaming_pull_future.result()
  except Exception as e:
    print(e)

In [ ]:
#create histogram of ending stock values for our mutliple simulations
plt.rcParams["figure.figsize"] = (9,8)
plt.hist(sim_result,bins=50)
plt.suptitle('Monte Carlo Simulation Results', fontsize=20)
plt.xlabel('Stock Value $')
plt.show()

#create histogram of ending protfolio values for our mutliple simulations, compare this value to "initial_capital"
plt.hist(portfolio_total,bins=50)
plt.suptitle('Portfolio Simulated Results (based on $100K investment / ' + str(trading_days) + ' days)', fontsize=20)
plt.xlabel('Portfolio Value $')
plt.show()